# Building an Intelligent Market Data Agent with Flotorch ADK

This notebook demonstrates how to build a powerful, data-aware agent using the `FlotorchADKAgent`. Our goal is to create a **Market Data Agent** that can intelligently retrieve and reason about cryptocurrency data.

### Objective

The agent will leverage the `FlotorchADKAgent` client to connect to an agent pre-configured on the Flotorch platform. This remote agent is equipped with **CoinGecko MCP tools** to perform several key functions:

* **Retrieve Data**: Fetch real-time and historical cryptocurrency data.
* **Perform Reasoning**: Analyze trends, make comparisons, and provide performance insights.
* **Use Context**: Automatically use the current date and time as a contextual anchor for all queries, ensuring accurate, date-aware data retrieval without needing extra tool calls.

### Key Advantage: Decoupling Configuration

This notebook's primary benefit is the **decoupling of configuration from code**. All the complex tool logic (CoinGecko API calls) and instructions are managed on the Flotorch platform UI. We can update the agent's data sources or reasoning capabilities without ever changing or redeploying this notebook's code.

### Prerequisites

Before running this notebook, you must have an agent configured in the Flotorch console (https://console.flotorch.cloud/).

1.  **Agent Name**: The agent must be named `market-data-agent` (or you must change the `AGENT_NAME` variable below).
2.  **Tools**: This agent must be configured in the UI with the necessary **CoinGecko MCP tools** to handle queries about prices, trends, and market data.


# Agent + Tool Configuration (Flotorch Console)

Use the following configuration to set up your **CoinGecko cryptocurrency data agent** inside the **Flotorch Console**.


## Step 1: Agent Configuration

Paste the following **YAML configuration** when creating your agent in the console.  
Attach the tools (listed in the next section) to this same agent.  
> ⚠️ **Do not modify the System Prompt** — paste it exactly as shown below.

```yaml
agent:
  name: market-data-agent
  systemPrompt: |
    You are a cryptocurrency data retrieval agent connected to the CoinGecko MCP server.

    You have access ONLY to the following tools:
    [get-simple-price, get-coins-markets, get-range-coins-market-chart,
      get-range-coins-ohlc, get-coins-history, get-search-trending]

    ---

    ### 🧭 BEHAVIOR RULES

    1. **USER DATE/TIME CONTEXT**
       - The user will always include today’s date and/or time.
       - Use that date as your base for all range or time-based calculations.
       - Never call or assume any “current date” outside what’s given.

    2. **TOOL SELECTION — STRICT**
       - Use exactly ONE tool per query.
       - Map user intent directly:
         - “Current price” → get-simple-price
         - “Top coins / market overview” → get-coins-markets
         - “Last few days / price trend” → get-range-coins-market-chart
         - “OHLC / candlestick data” → get-range-coins-ohlc
         - “Specific historical date” → get-coins-history
         - “Trending coins” → get-search-trending
       - Do NOT call tools not listed here.
       - Do NOT call multiple tools for a single user query.

    3. **PARAMETER CONTROL (STRICT TYPES)**
       - Validate all required parameters before calling.
       - Required fields:
         • id — coin name/id, lowercase (e.g., “bitcoin”, “ethereum”)  
         • vs_currency — “usd” unless otherwise stated  
         • from / to — ISO date strings (e.g., “2025-10-30”, “2025-11-06”)  
         • interval — “daily” for range data  
         • precision — always "2" (string, not integer)  
         • per_page — "10"  
         • page — "1"  
       - Never send invalid types — especially `precision`, which must be a string.

    4. **PAYLOAD MINIMIZATION**
       - Limit ranges to a maximum of 7 days unless otherwise specified.
       - Use interval="daily" not "hourly".
       - Use pagination (per_page="10", page="1") and fetch only one page unless user explicitly says “next” or “more”.
       - Prefer concise jq filters to extract only required data fields (e.g., .prices or .market_caps).

    5. **ERROR HANDLING**
       - If a tool call fails due to validation (e.g., bad type or missing field):
         • Correct the parameter once and retry.  
         • If it still fails, stop and explain the reason clearly.  
       - Never switch to another tool after an error.

    6. **OUTPUT FORMAT**
       - Respond in one concise paragraph, in clear human-readable form.
       - Include only key numbers: prices, % changes, or trend summary.
       - Do not output JSON, code, or raw tables.
       - Example:
         “As of 2025-11-06, Ethereum’s average daily price ranged between $3,190 and $3,250 over 
         the last 7 days, showing a mild uptrend.”

    7. **ANALYSIS RULES**
       - If user asks about “trend”, “performance”, or “compare”, summarize direction (up/down/flat) based on limited data.
       - Do not overanalyze or fetch large payloads.

    8. **STOPPING CRITERIA**
       - Once a valid response is obtained, stop all tool calls.
       - Never loop, never “try something else” after success.

    ---

    ### ⚙️ PARAMETER SANITY EXAMPLES

    Examples for correct tool usage:

    - **BTC price today:** Tool → get-simple-price  
      Params → {"ids": "bitcoin", "vs_currencies": "usd"}

    - **ETH trend last 7 days:** Tool → get-range-coins-market-chart  
      Params → {"id": "ethereum", "from": "2025-10-30", "to": "2025-11-06", "vs_currency": "usd",
                "interval": "daily", "precision": "2", "jq_filter": ".prices"}

    - **BTC OHLC data last week:** Tool → get-range-coins-ohlc  
      Params → {"id": "bitcoin", "vs_currency": "usd", "days": "7", "interval": "daily", "precision": "2"}

    - **Dogecoin on 2024-01-01:** Tool → get-coins-history  
      Params → {"id": "dogecoin", "date": "2024-01-01", "localization": "false"}

    - **Trending coins today:** Tool → get-search-trending  
      Params → {}

    ---

    ### 🧩 SUMMARY

    Your job:
    - Choose exactly one correct tool.  
    - Pass type-safe, minimal parameters.  
    - Retry once if a validation error occurs.  
    - Return one short paragraph that summarizes the result naturally.
  goal: |
    Use CoinGecko MCP tools to deliver accurate, time-specific cryptocurrency data 
    using only the approved tools. 
    Always rely on the date/time provided by the user and never attempt to infer or fetch the current date. 

    Prioritize small, precise, and date-aware API calls with correct parameters and minimal payloads. 
    Use only one tool per user query and stop after one retry if an error occurs. 
    All outputs must be concise natural-language summaries — no JSON or code.
```

## Tool Configuration Instructions

Before running this notebook, please configure the following tools in your **Console → Tools** section.

Each tool corresponds to a specific **CoinGecko API endpoint**, but they all share the **same transport and URL**.


### Common Configuration (Same for All Tools)

- **Transport:** `HTTPStream`  
- **URL:** `https://mcp.api.coingecko.com/mcp`


### Tools to Add

Create each tool **separately** using the values below.  

| Tool Name | Description |
|------------|--------------|
| `get-simple-price` | Retrieve simple cryptocurrency prices for specified coins and currencies. |
| `get-coins-markets` | Fetch market data (price, volume, market cap, etc.) for multiple coins. |
| `get-range-coins-market-chart` | Fetch market chart data for a specific coin within a date range. |
| `get-range-coins-ohlc` | Retrieve OHLC (Open, High, Low, Close) data for a specific coin over time. |
| `get-coins-history` | Get historical market data (price, market cap, etc.) for a coin on a specific date. |
| `get-search-trending` | Retrieve currently trending search coins on CoinGecko. |


### Example Setup

1. Go to **Console → Tools → Add Tool**
2. Enter:
   - **Name:** `get-simple-price`  
   - **Description:** Retrieve simple cryptocurrency prices for specified coins and currencies.  
   - **Transport:** `HTTPStream`  
   - **URL:** `https://mcp.api.coingecko.com/mcp`
3. Click **Save**
4. Repeat the same process for each tool listed above.


### Note

All tools use the same **CoinGecko MCP base URL** and **HTTPStream transport**, but represent **different API functionalities**.  
After configuring them, you’ll need to **pass all tool names** to the agent during creation so that your agent can access each endpoint as needed.

## 1. Environment Setup

First, we'll install the required packages and define our API credentials and configuration variables.

In [ ]:
# Install the Flotorch ADK package
%pip install flotorch[adk]

In [ ]:
# === CONFIGURATION ===
# Replace with your Flotorch credentials and agent details

FLOTORCH_API_KEY = "<your-flotorch-api-key>"
FLOTORCH_BASE_URL = "<your-flotorch-gateway-base-url>" # e.g., https://gateway.flotorch.cloud
AGENT_NAME = "market-data-agent" # The agent_name you configured in the Flotorch UI
APP_NAME = "market_data_app"
USER_ID = "crypto_user_001"

In [ ]:
# === IMPORTS ===
from datetime import datetime

from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types

print("Imported necessary libraries successfully")

## 2. Context-Aware Query Formatting

To ensure our agent *always* has the correct temporal context, we'll create helper functions. `format_query` automatically finds the current date and time and appends it to the user's query. This instructs the agent to use this specific date as the anchor for any data retrieval.

This is a key part of our agent's "data-aware reasoning."

In [ ]:
def get_current_datetime() -> str:
    """Get the current date and time in a formatted string."""
    now = datetime.now()
    return f"Current date: {now.strftime('%Y-%m-%d')} (format: YYYY-MM-DD), Current time: {now.strftime('%H:%M:%S')}, Day: {now.strftime('%A')}"

def format_query(query: str) -> str:
    """Appends the current date and time context to the user query."""
    current_date = get_current_datetime()
    final_query = f"{query}. For context, today's date is: {current_date}"
    return final_query

## 3. Initializing the Remote Agent

Here, we create an instance of the `FlotorchADKAgent` client. We provide the `AGENT_NAME` and our API credentials. The client connects to the Flotorch platform and fetches the complete agent configuration, including its instructions and the CoinGecko MCP tools.

In [ ]:
flotorch_client = FlotorchADKAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

# Initialize the Flotorch agent
agent = flotorch_client.get_agent()
print(f"FlotorchADKAgent '{agent.name}' created successfully.")

## 4. Configuring the Runner

Next, we configure the `Runner`. This component orchestrates the interaction. We provide it with our remotely-fetched `agent` and a `FlotorchADKSession` service to handle the conversation's short-term memory.

In [ ]:
# Initialize Session Service
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

# Initialize Runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

print("Runner configured successfully for FlotorchADKAgent.")

## 5. Interactive Chat Functions

These functions handle the turn-by-turn conversation with the agent. `run_single_turn` sends the message and processes the event stream to get a final answer. `chat_with_agent` is our simple async wrapper.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    """
    Send a message to the FlotorchADKAgent and get the response.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that request."

async def chat_with_agent(query, session_id):
    """
    Chat function that takes a user query and returns the FlotorchADKAgent's response.
    """
    # Use our formatter to inject date-time context
    formatted_query = format_query(query)
    response = run_single_turn(formatted_query, session_id, USER_ID, runner)
    return response

print("Chat function ready. Use chat_with_agent('your query', session.id) to interact.")

## 6. Running the Market Data Agent

With the setup complete, we can now interact with our agent. We'll first create a new session, then ask it questions based on our defined use cases.

*(Note: You must `await` the function calls, as this notebook environment supports top-level await. If yours does not, wrap them in an `asyncio.run()` block.)*

In [ ]:
# Create a new session for our conversation
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)

print(f"New session created with ID: {session.id}")

### Use Case 1:

In [ ]:
query1 = "What is the price of Bitcoin today?"
response1 = await chat_with_agent(format_query(query1), session.id)
print(f"\n\nUser Query: {query1}")
print(f"\nAgent Response: {response1}")

### Use Case 2:

In [ ]:
query2 = "summarize how the crypto market trended over the last week — rising or falling overall"
response2 = await chat_with_agent(format_query(query2), session.id)
print(f"User Query: {query2}")
print(f"\nAgent Response: {response2}")

## Interactive Chat

In [ ]:
# ===  Interactive Chatbot Loop ===
# Chat with the agent in a loop. Type 'exit' to quit.

print("\n=== Interactive Chat Started ===")
print("Type your message. Type 'exit' to quit.\n")

while True:
    user_msg = input("You: ").strip()
    if not user_msg:
        continue
    if user_msg.lower() in {"exit", "quit", "q"}:
        break

    try:
        user_query = format_query(user_msg)
        agent_reply = await chat_with_agent(user_query, session.id)
        print(f"user: {user_msg}")
        print(f"\nAgent: {agent_reply}\n")
    except Exception as e:
        print(f"Error: {e}\n")


## Summary

This notebook successfully demonstrated how to build a powerful data retrieval and reasoning agent using the Flotorch ADK framework.

By combining the `FlotorchADKAgent` with an agent configured with CoinGecko MCP tools, we achieved:

- **Simplified Codebase**: We instantiated a fully functional agent with minimal code, as its core logic (persona, instructions, and tool definitions) resides on the Flotorch platform.
- **Dynamic Contextual Understanding**: By automatically injecting the current date and time, we ensured the agent's tool calls are always accurate and relevant.
- **Data-Aware Reasoning**: The agent was able to field complex queries about market trends and comparisons, not just simple data lookups.
- **Agile Development**: We can now update the agent's tools (e.g., add new data sources) or reasoning instructions directly in the Flotorch UI without any code changes, accelerating the development lifecycle.